In [1]:
import pickle
import numpy as np
import pandas as pd
import re

seed = 2023

In [2]:
import os
os.chdir("../../")

In [3]:
from nlpsig_networks.scripts.ffn_baseline_functions import (
    histories_baseline_hyperparameter_search
)

/storage/ttseriotou/nlpsig-networks-env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Rumours

In [4]:
%run notebooks/Rumours/load_sbert-embeddings.py

In [5]:
df_rumours.head()

,id,label,datetime,text,timeline_id,set
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0,train
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0,train
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0,train
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0,train
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0,train


In [6]:
sbert_embeddings.shape

torch.Size([5568, 384])

In [27]:
from nlpsig_networks.scripts.ffn_baseline_functions import obtain_signatures_history

x_data = obtain_signatures_history(method="umap",
                              dimension=10,
                              sig_depth=3,
                              log_signature=True,
                              df=df_rumours,
                              id_column="timeline_id",
                              label_column="label",
                              embeddings=sbert_embeddings.numpy(),
                              seed=seed,
                              path_indices=None,
                              concatenate_current=False)

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


100%|██████████| 5568/5568 [01:10<00:00, 78.87it/s] 


In [32]:
x_data.shape

torch.Size([5568, 1110])

# Baseline: FFN using signatures

In [7]:
dim_reduce_methods = ["umap", "gaussian_random_projection"]
num_epochs = 100
batch=64
log_sig = True
log_signature_dimensions_and_sig_depths = [(28, 2), (10, 3)]#[(28, 2), (10, 3), (6, 4)]
hidden_dim_sizes = [[64,64],[128,128], [256,256]]#[[64,64], [128,128], [256,256]]
dropout_rates =  [0.2, 0.1]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [0, 1, 12, 123, 1234]
loss = "focal"
gamma = 2
patience=5
validation_metric = "f1"

In [8]:
ffn_logsignature_umap, best_ffn_logsignature_umap, _, __ = histories_baseline_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings.numpy(),
    y_data=y_data,
    output_dim=output_dim,
    hidden_dim_sizes=hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    use_signatures=True,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    batch_size=batch,
    log_signature=log_sig,
    dim_reduce_methods=dim_reduce_methods,
    dimension_and_sig_depths=log_signature_dimensions_and_sig_depths, 
    path_indices=None,
    #data_split_seed=0,
    split_ids= None, #torch.tensor(df_rumours['timeline_id'].astype(int)),
    split_indices = (df_rumours[df_rumours['set']=='train'].index, df_rumours[df_rumours['set']=='dev'].index, df_rumours[df_rumours['set']=='test'].index),
    k_fold=False,
    patience=patience,
    validation_metric=validation_metric,
    results_output=None,
    verbose=False
)

  0%|          | 0/2 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.





































































































































































































































































































































































































































































































































































































































































100%|██████████| 5568/5568 [01:14<00:00, 75.15it/s]















100%|██████████| 3/3 [05:05<00:00, 101.73s/it]














100%|██████████| 3/3 [03:08<00:00, 62.96s/it]


100%|██████████| 2/2 [08:14<00:00, 247.05s/it]















100%|██████████| 3/3 [07:37<00:00, 152.41s/it]














100%|██████████| 3/3 [04:23<00:00, 87.86s/it]


100%|█████

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.





































































































































































































































































































































































































































































































































































































































100%|██████████| 5568/5568 [01:12<00:00, 77.12it/s]















100%|██████████| 3/3 [04:22<00:00, 87.45s/it]














100%|██████████| 3/3 [04:14<00:00, 84.81s/it]


100%|██████████| 2/2 [08:36<00:00, 258.39s/it]















100%|██████████| 3/3 [06:07<00:00, 122.61s/it]














100%|██████████| 3/3 [04:46<00:00, 95.41s/it]


100%|██████████| 2/2 [10:54<00:00, 327.03s/

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


100%|██████████| 5568/5568 [01:13<00:00, 76.03it/s] 


In [9]:
ffn_logsignature_umap

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,gamma,k_fold,n_splits,batch_size,model_id,input_dim,dimension,sig_depth,method,log_signature
0,focal,0.459485,0.432685,"[0.5559906029757243, 0.3093788063337394]",0.441638,"[0.4876373626373626, 0.3956386292834891]",0.450315,"[0.6466302367941712, 0.254]",0.237917,0.647687,...,2,False,None,64,0.00,1196,28,2,umap,True
0,focal,0.523356,0.343554,"[0.6871088861076345, 0.0]",0.261678,"[0.5233555767397522, 0.0]",0.500000,"[1.0, 0.0]",0.240476,0.637011,...,2,False,None,64,0.00,1196,28,2,umap,True
0,focal,0.523356,0.505189,"[0.41037735849056606, 0.6]",0.540970,"[0.5819397993311036, 0.5]",0.533470,"[0.31693989071038253, 0.75]",0.229389,0.622776,...,2,False,None,64,0.00,1196,28,2,umap,True
0,focal,0.513823,0.456377,"[0.2796610169491526, 0.6330935251798561]",0.558512,"[0.6226415094339622, 0.4943820224719101]",0.530164,"[0.18032786885245902, 0.88]",0.236081,0.665480,...,2,False,None,64,0.00,1196,28,2,umap,True
0,focal,0.535748,0.527329,"[0.46424642464246424, 0.5904121110176619]",0.547773,"[0.5861111111111111, 0.5094339622641509]",0.543168,"[0.3843351548269581, 0.702]",0.234094,0.704626,...,2,False,None,64,0.00,1196,28,2,umap,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,focal,0.648236,0.626693,"[0.7163720215219063, 0.5370138017565872]",0.670110,"[0.6196808510638298, 0.7205387205387206]",0.638408,"[0.848816029143898, 0.428]",0.269243,0.633452,...,2,False,None,64,1.11,1494,10,3,umap,True
0,focal,0.622498,0.603952,"[0.6896551724137931, 0.5182481751824817]",0.633359,"[0.6052269601100413, 0.6614906832298136]",0.613729,"[0.8014571948998178, 0.426]",0.236628,0.665480,...,2,False,None,64,1.11,1494,10,3,umap,True
0,focal,0.555767,0.550172,"[0.5, 0.6003430531732418]",0.566940,"[0.608355091383812, 0.5255255255255256]",0.562204,"[0.424408014571949, 0.7]",0.225432,0.665480,...,2,False,None,64,1.11,1494,10,3,umap,True
0,focal,0.555767,0.538836,"[0.6272000000000001, 0.4504716981132076]",0.554026,"[0.5592011412268189, 0.5488505747126436]",0.548013,"[0.7140255009107468, 0.382]",0.269061,0.590747,...,2,False,None,64,1.11,1494,10,3,umap,True


In [15]:
best_ffn_logsignature_umap

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,gamma,k_fold,n_splits,batch_size,input_dim,dimension,sig_depth,method,log_signature
0,focal,0.524309,0.514912,"[0.44739756367663347, 0.5824267782426777]",0.535670,"[0.5706214689265536, 0.5007194244604316]",0.531971,"[0.3679417122040073, 0.696]",0.220502,0.711744,...,0,2,False,None,64,1196,28,2,umap,True
0,focal,0.524309,0.483157,"[0.33731739707835323, 0.6289962825278811]",0.561570,"[0.6225490196078431, 0.5005917159763313]",0.538665,"[0.23132969034608378, 0.846]",0.235918,0.548043,...,1,2,False,None,64,1196,28,2,umap,True
0,focal,0.527169,0.511991,"[0.425925925925926, 0.5980551053484602]",0.543426,"[0.5841269841269842, 0.502724795640327]",0.536577,"[0.33515482695810567, 0.738]",0.226550,0.697509,...,12,2,False,None,64,1196,28,2,umap,True
0,focal,0.522402,0.508569,"[0.42611683848797244, 0.5910204081632653]",0.536692,"[0.5740740740740741, 0.4993103448275862]",0.531399,"[0.33879781420765026, 0.724]",0.236236,0.651246,...,123,2,False,None,64,1196,28,2,umap,True
0,focal,0.519542,0.517331,"[0.48466257668711654, 0.55]",0.524611,"[0.5524475524475524, 0.4967741935483871]",0.523847,"[0.43169398907103823, 0.616]",0.232126,0.683274,...,1234,2,False,None,64,1196,28,2,umap,True


In [10]:
best_ffn_logsignature_umap["valid_f1"].mean()

0.6404925294109669

In [11]:
best_ffn_logsignature_umap["f1"].mean()

0.5071918876138286

In [13]:
best_ffn_logsignature_umap["precision"].mean()

0.5403939573636072

In [14]:
best_ffn_logsignature_umap["recall"].mean()

0.5324918032786885

In [12]:
np.stack(best_ffn_logsignature_umap["f1_scores"]).mean(axis=0)

array([0.42428406, 0.59009971])

In [30]:
import signatory

signatory.logsignature_channels(10,3)

385

In [22]:
406 + 384 + 406

1196

In [31]:
385 * 2 + 384

1154